Based on the previous Programming Assignment: "ETL - from Smartphone to ObjectStore" the smartphone will create a accelarometer sensor dataset and use NodeRED to store it in Apache CouchDB.

This assignment is learn how to classify a dataset.
Steps:
1) Using the data generation pipline two different movement patterns will be recorded.
2) Using NodeRED change the CLASS to "0" for one case and to "1" for the other case.
3) Train and tune the Apache SparkML classifier


In [3]:
#your cloudant credentials go here
# @hidden_cell
credentials_1 = {
  'password':""""",
  'custom_url':'',
  'username':'',
  'url':'https://undefined'
}


Let's create a SparkSession object and put the Cloudant credentials into it

In [4]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [6]:
df=spark.read.load('shake_classification', "org.apache.bahir.cloudant")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|
|    0|velqwert|-0.01|-0.01|-0.01|020533cb000ab3de5...|1-df37baeaa4c3926...|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|
|    0|velqwert| 0.01| 0.01| 0.01|020533cb000ab3de5...|1-3e653d871d6c029...|
|    0|velqwert|-0.02|-0.02|-0.02|020533cb000ab3de5...|1-f3902329a0f1765...|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|
|    0|velqwert| 0.01| 0.01| 0.01|020533cb000ab3de5...|1-3e653d871d6c029...|
|    0|velqwert|-0.01|-0.01|-0.01|020533cb000ab3de5...|1-df37baeaa4c3926...|

Please create a VectorAssembler which consumed columns X, Y and Z and produces a column “features”


In [11]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                 outputCol="features")

Please insatiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [15]:
from pyspark.ml.classification import GBTClassifier

classifier = GBTClassifier(labelCol="CLASS", featuresCol="features",maxIter=10)



Let’s train and evaluate…


In [16]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [17]:
model = pipeline.fit(df)

In [18]:
prediction = model.transform(df)

In [19]:
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|          (3,[],[])|       0.0|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|          (3,[],[])|       0.0|
|    0|velqwert|-0.01|-0.01|-0.01|020533cb000ab3de5...|1-df37baeaa4c3926...|[-0.01,-0.01,-0.01]|       0.0|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|          (3,[],[])|       0.0|
|    0|velqwert|  0.0|  0.0|  0.0|020533cb000ab3de5...|1-f5e2391b0458648...|          (3,[],[])|       0.0|
|    0|velqwert| 0.01| 0.01| 0.01|020533cb000ab3de5...|1-3e653d871d6c029...|   [0.01,0.01,0.01]|       0.0|
|    0|velqwert|-0.02|-0.02|

In [21]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction) 

0.9485074626865672

 If the value is greater than 0.55 means that we are performing better than random guesses.


# Project submission

Please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Courera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera.

In [22]:
!rm -Rf a2_m2.json

In [23]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [24]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-10-11 17:01:32--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2029 (2.0K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,029       --.-K/s   in 0s      

2018-10-11 17:01:32 (15.8 MB/s) - ‘rklib.py’ saved [2029/2029]



In [25]:
!zip -r a2_m2.json.zip a2_m2.json

  adding: a2_m2.json/ (stored 0%)
  adding: a2_m2.json/.part-00000-47e94feb-efd1-463d-8c46-789498b17308.json.crc (stored 0%)
  adding: a2_m2.json/_SUCCESS (stored 0%)
  adding: a2_m2.json/._SUCCESS.crc (stored 0%)
  adding: a2_m2.json/part-00000-47e94feb-efd1-463d-8c46-789498b17308.json (deflated 89%)


In [26]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [28]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = ""
secret = "kUWz12xEQLhECyVJ"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
